Base에서 실행

# Textcuboid


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string
import keras
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
train_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/train.csv')
test_df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/ag_news_dataset/test.csv')

train_df.head(10)

TEXT_LABELS = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}

def combine_title_and_description(df):
    # Returns a dataset with the title and description fields combined
    df['text'] = df[['Title', 'Description']].agg('. '.join, axis=1)
    df = df.drop(['Title', 'Description'], axis=1)
    return df

train_df = combine_title_and_description(train_df)
test_df = combine_title_and_description(test_df)

#각 클래스별로 5000개씩 총 2만개의 데이터를 샘플랭(너무 크면 TextCuboid의 용량이 너무 커진다)
sampled_df = train_df.groupby("Class Index").apply(lambda x: x.sample(5000, random_state=10))

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    #text = re.sub('\n', '', text)
    #text = re.sub('\w*\d\w*', '', text)
    return text

#전처리 특수기호 없애기
sampled_df['text']=sampled_df['text'].apply(clean_text)

sampled_df = sampled_df.reset_index(drop=True)

train_df = sampled_df.groupby("Class Index").apply(lambda x: x.sample(4000, random_state=10))
train_idx = [x[1] for x in train_df.index]
test_df = sampled_df.drop(train_idx)

x_train=list(train_df['text'])
y_train=list(train_df['Class Index'])
x_test=list(test_df['text'])
y_test=list(test_df['Class Index'])

to_txt_filter=x_train+x_test
y=list(y_train)+list(y_test)

In [3]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=label[:16000]
y_test=label[16000:]

In [4]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 176

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

## 2) 어텐션 레이어 정의

In [6]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads=8):
        super(MultiHeadAttention, self).__init__()
        self.embedding_dim = embedding_dim # d_model
        self.num_heads = num_heads

        assert embedding_dim % self.num_heads == 0

        self.projection_dim = embedding_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embedding_dim)
        self.key_dense = tf.keras.layers.Dense(embedding_dim)
        self.value_dense = tf.keras.layers.Dense(embedding_dim)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def scaled_dot_product_attention(self, query, key, value):
        matmul_qk = tf.matmul(query, key, transpose_b=True)
        depth = tf.cast(tf.shape(key)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)
        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, value)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]

        # (batch_size, seq_len, embedding_dim)
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        # (batch_size, num_heads, seq_len, projection_dim)
        query = self.split_heads(query, batch_size)  
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = self.scaled_dot_product_attention(query, key, value)
        # (batch_size, seq_len, num_heads, projection_dim)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  

        # (batch_size, seq_len, embedding_dim)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embedding_dim))
        outputs = self.dense(concat_attention)
        return outputs


class TransformerBlock(tf.keras.layers.Layer):
    
        
    def __init__(self, embedding_dim, num_heads, dff, rate=0.1, **kwargs):
        super(TransformerBlock, self).__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_heads = num_heads
        self.dff = dff
        self.rate = rate
        self.att = MultiHeadAttention(embedding_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(dff, activation="relu"),
             tf.keras.layers.Dense(embedding_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) # 첫번째 서브층 : 멀티 헤드 어텐션
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Add & Norm
        ffn_output = self.ffn(out1) # 두번째 서브층 : 포지션 와이즈 피드 포워드 신경망
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Add & Norm
    
    def get_config(self):
        config = super(TransformerBlock, self).get_config()
        config.update({
            'embedding_dim': self.embedding_dim,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'rate': self.rate
        })
        return config


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_len, vocab_size, embedding_dim, **kwargs):
        super(TokenAndPositionEmbedding, self).__init__(**kwargs)
    
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)

    def call(self, x):
        max_len = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_len, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        config = super(TokenAndPositionEmbedding, self).get_config()
        config.update({
            'max_len': self.max_len,
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim
        })
        return config

## 3) textcuboid 분류

In [7]:
textcuboid=np.load('./1-Channel textcuboid_ag(elmo).npy')

In [8]:
textcuboid_test=np.load('./1-Channel textcuboid_test_ag(elmo).npy')

In [9]:
tmp = [[x,y,z] for x, y, z in zip(xtext_train, textcuboid, y_train)]
import random
random.shuffle(tmp)
xtext_train = [n[0] for n in tmp]
textcuboid = [n[1] for n in tmp]
y_train = [n[2] for n in tmp]
xtext_train=np.array(xtext_train)
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [10]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_val=y_train[:1000]
y_train=y_train[1000:]


text_test=xtext_test
y_test=np.array(y_test)

In [11]:
import keras
from tensorflow.keras import layers

In [35]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, verbose=1),
]

In [36]:
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention

embedding_dim = 512  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 32  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기

Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)
transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
Text_x = transformer_block(Text_x)
Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
Text_attention_layer = Attention()
Text_a=Text_attention_layer([Text_x,Text_x])

ELMo_input = Input(shape=(46, 256))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
ELMo_x = Dense(256, activation='relu')(pooling)
ELMo_attention_layer = Attention()
ELMo_a=ELMo_attention_layer([ELMo_x,ELMo_x])


x = layers.concatenate([ELMo_x,ELMo_a, Text_x,Text_a], axis=-1)
x = Dropout(0.5)(x)
output_layer = Dense(4, activation='softmax')(x)

model = Model(inputs=[ELMo_input, Text_inputs], outputs=output_layer)

In [37]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [38]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 176)]        0           []                               
                                                                                                  
 token_and_position_embedding_4  (None, 176, 512)    20072960    ['input_9[0][0]']                
  (TokenAndPositionEmbedding)                                                                     
                                                                                                  
 input_10 (InputLayer)          [(None, 46, 256)]    0           []                               
                                                                                                  
 transformer_block_4 (Transform  (None, 176, 512)    1085984     ['token_and_position_embedd

In [39]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=30,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/30
59/59 [==============================] - 5s 69ms/step - loss: 1.0676 - accuracy: 0.6664 - val_loss: 0.3767 - val_accuracy: 0.8680 - lr: 0.0010
Epoch 2/30
59/59 [==============================] - 4s 65ms/step - loss: 0.3851 - accuracy: 0.8807 - val_loss: 0.3396 - val_accuracy: 0.8830 - lr: 0.0010
Epoch 3/30
59/59 [==============================] - 4s 63ms/step - loss: 0.3482 - accuracy: 0.8915 - val_loss: 0.2967 - val_accuracy: 0.9010 - lr: 0.0010
Epoch 4/30
59/59 [==============================] - 4s 63ms/step - loss: 0.3265 - accuracy: 0.8987 - val_loss: 0.2808 - val_accuracy: 0.9050 - lr: 0.0010
Epoch 5/30
59/59 [==============================] - 3s 52ms/step - loss: 0.2971 - accuracy: 0.9083 - val_loss: 0.3295 - val_accuracy: 0.8890 - lr: 0.0010
Epoch 6/30
59/59 [==============================] - 4s 63ms/step - loss: 0.3242 - accuracy: 0.9021 - val_loss: 0.2793 - val_accuracy: 0.9010 - lr: 0.0010
Epoch 7/30
59/59 [==============================] - 4s 63ms/step - loss: 0.2

In [40]:
model.evaluate([textcuboid_test,text_test],y_test)

125/125 [==============================] - 1s 5ms/step - loss: 0.1687 - accuracy: 0.9503


[0.1686539500951767, 0.9502500295639038]

In [41]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)

125/125 [==============================] - 1s 4ms/step - loss: 0.1891 - accuracy: 0.9385


[0.18911370635032654, 0.9384999871253967]

In [27]:
np.average([95.2,95.4,95.2])

95.26666666666667